In [1]:
from package import *

import requests
from pprint import pprint
from functools import partial
import pandas
import matplotlib.pyplot as plt
import os
import sys

github_folder = os.path.join(os.getenv('USERPROFILE'), 'Documents', 'Github', 'python-youtube-api')
sys.path.append(github_folder)

pprint = partial(pprint, width = 120)

#youtube = YouTubeDatabase(github.youtube_api_key, 'youtube_database')
killian = "UCjdQaSJCYS4o2eG93MvIwqg"
test_video = "836Lv_swaCk"
roosterteeth = 'UCzH3iADRIq1IJlIXjfNgTpA'
vs_playlist = "PLTur7oukosPH7Kvex9-rUiZgVIVGt6C--"
youtube = YouTube(github.youtube_api_key)
youtubedb = YoutubeDatabase(youtube, 'youtube_database')
print("Finished!")


Finished!


In [2]:

subscriptions = github.youtube_subscriptions
youtubedb.importChannels([v for k,v in sorted(subscriptions.items())])

Importing 'A+Start' with 447 videos into the database.


 83% (374 of 447) |###################    | Elapsed Time: 0:00:00 ETA:  0:00:00

Importing 'AI and Games' with 167 videos into the database.


 79% (132 of 167) |##################     | Elapsed Time: 0:00:00 ETA:  0:00:00

Importing 'Abroad in Japan' with 129 videos into the database.


...

 99% (118 of 119) |###################### | Elapsed Time: 0:00:22 ETA:  0:00:00

Importing 'videogamedunkey' with 577 videos into the database.


 99% (576 of 577) |###################### | Elapsed Time: 0:01:52 ETA:  0:00:00

Importing '❀ rachie/れいち ❀' with 111 videos into the database.


 99% (110 of 111) |###################### | Elapsed Time: 0:00:22 ETA:  0:00:00

In [ ]:
youtubedb.importChannel(killian)
print("Finished!")

In [ ]:
pprint(vs.items[0].content_details)

In [ ]:
print(PlaylistItemResource(vs.items[0].response))